# Define entropy

In [4]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_excel("hypothetical data.xlsx",sheet_name="static")
df.head()

,y,x1,x2,x3,x4,c
0,209.243988,10.512692,20.299629,2.572243,56.094400,2
1,180.813118,9.443407,18.284721,4.570159,47.547308,1
2,215.933346,10.924296,20.512234,4.641844,53.150497,2
3,191.899247,9.694745,18.797090,4.472520,45.323743,2
4,250.048339,10.670259,23.214880,5.402460,64.529068,1


In [ ]:
import math
c = len(df.c.unique())

def func_I(T): # equation 3
    I = 0
    for i in range(1,c):
        I += p[i] * math.log2(p[i])
    return (-1)*I

def func_E(a): # equation 4
    E = 0
    for j in range(1,v):
        E += q[j,a]*I[j,a]
    return E

def func_I(j,a): # equation 5
    p[i,j] = s[i,j,a] / S[j,a]
    I[j,a] = 0
    for i in range(1,c):
        I[j,a] += p[i,j] * math.log2(p[i,j])
    return (-1)*I

def func_s(T_Independent_Variables,):
    features_list = list(T_Independent_Variables.columns)
    v = len(features_list)
    s = []
    for a in features_list:
        for i in range(1,c):
            for j in range(1,df[a].unique()):
                print(a,i,j)


def func_S(j,a):
    S[j,a] = 0
    for i in range(1,c):
        S[j,a] += s[i,j,a]


def func_Gain(T,a):
    gain = func_I(T) - func_E(a)
    return gain

In [17]:
df.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'c'], dtype='object')

In [24]:
df.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'c'], dtype='object')

In [32]:
T_Independent_Variables = df.drop(['y', 'c'], axis=1)

features_list = list(T_Independent_Variables.columns)

In [34]:
df[features_list[0]].unique()


array([10.51269165,  9.44340698, 10.92429621,  9.69474542, 10.6702591 ,
        8.6310664 ,  8.94981555,  9.3830834 , 10.98411767,  9.64846082,
       11.45544043, 10.08958885,  9.95678344,  9.7763889 ,  9.46389739,
       10.67079475,  8.76368597, 10.29122375,  8.40181907, 10.32773236,
        8.29892742,  9.58043913, 11.55653171, 10.49417404, 11.27085563,
        9.23562422,  9.89744586,  8.89186919, 11.00165327,  9.66404997,
        9.05935341, 10.19602014, 10.10729666, 10.55183168, 10.29357186,
        9.62835955,  9.7386339 , 10.81293639,  9.95658923, 10.24550276,
        8.24153379, 10.98626031,  7.97153356,  9.88717417,  9.37381927,
       10.06014661, 11.74454434,  9.58539241,  8.91055207,  8.9131957 ,
        9.35895246, 11.3336302 , 11.30258772,  9.57927423, 10.89409625,
        8.34545824,  9.14128232, 12.52134972,  9.89498468, 11.70124094,
       11.36647488,  9.10016704,  9.26682103,  9.05350291,  9.98259905,
       11.75461654, 10.32967792,  8.5447581 , 10.19377192,  9.73